In [36]:
import numpy as np
import pandas as pd
import math
import csv
import jieba

In [37]:
def load_data(file_name):
    joins = []
    predicates = []
    tables = []
    label = []

    # Load queries
    with open(file_name + ".csv", 'rU') as f:
        data_raw = list(list(rec) for rec in csv.reader(f, delimiter='#'))
        for row in data_raw:
            tables.append(row[0].split(','))
            joins.append(row[1].split(','))
            predicates.append(row[2].split(','))
            if int(row[3]) < 1:
                print("Queries must have non-zero cardinalities")
                exit(1)
            label.append(row[3])
    print("Loaded queries")
    
    return joins, predicates, tables, label

In [38]:
train_joins,train_predicates,train_tables,train_label = load_data("train")
train_tables

Loaded queries


C:\Users\DELL\AppData\Local\Temp/ipykernel_17720/3287742540.py:8: DeprecationWarning: 'U' mode is deprecated
  with open(file_name + ".csv", 'rU') as f:


[['title t', 'movie_companies mc', 'movie_info mi'],
 ['title t', 'movie_info mi', 'movie_info_idx mi_idx'],
 ['title t', 'movie_info mi', 'movie_info_idx mi_idx'],
 ['movie_keyword mk'],
 ['title t', 'cast_info ci'],
 ['title t', 'movie_info mi', 'movie_keyword mk'],
 ['title t', 'movie_keyword mk'],
 ['title t', 'movie_companies mc', 'movie_info_idx mi_idx'],
 ['title t', 'movie_companies mc', 'movie_info mi'],
 ['cast_info ci'],
 ['title t', 'cast_info ci', 'movie_keyword mk'],
 ['title t', 'movie_info mi'],
 ['cast_info ci'],
 ['title t', 'cast_info ci', 'movie_keyword mk'],
 ['title t', 'cast_info ci'],
 ['title t', 'cast_info ci'],
 ['title t', 'movie_info_idx mi_idx', 'movie_keyword mk'],
 ['movie_keyword mk'],
 ['title t', 'movie_info mi', 'movie_keyword mk'],
 ['movie_keyword mk'],
 ['title t', 'movie_companies mc', 'cast_info ci'],
 ['cast_info ci'],
 ['title t', 'movie_info mi'],
 ['title t', 'movie_keyword mk'],
 ['title t', 'cast_info ci', 'movie_info mi'],
 ['title t', 'm

In [39]:
jieba.load_userdict("dict.txt")
interpunctuations=['#','##',' ','title','t','movie_companies','mc','cast_info','ci','movie_info','mi','movie_info_idx','mi_idx','movie_keyword','mk']
table_name = ['title t','movie_companies mc','cast_info ci','movie_info mi','movie_info_idx mi_idx','movie_keyword mk']

In [40]:
joins_set = set()
for line in train_joins:
    for item in line:
        joins_set.add(item)
joins_set = list(joins_set)
joins_set

['',
 't.id=mi_idx.movie_id',
 't.id=mk.movie_id',
 't.id=mc.movie_id',
 't.id=ci.movie_id',
 't.id=mi.movie_id']

In [41]:
table_features = []
link_features = []
for line in range(len(train_joins)):
    table_vector = np.zeros(len(table_name))
    link_vector = np.zeros(len(joins_set))
    for item in train_tables[line]:
        id = table_name.index(item)
        table_vector[id] = 1
    for item in train_joins[line]:
        id = joins_set.index(item)
        link_vector[id] = 1
    table_features.append(table_vector)
    link_features.append(link_vector)

In [45]:
table_features

[array([1., 1., 0., 1., 0., 0.]),
 array([1., 0., 0., 1., 1., 0.]),
 array([1., 0., 0., 1., 1., 0.]),
 array([0., 0., 0., 0., 0., 1.]),
 array([1., 0., 1., 0., 0., 0.]),
 array([1., 0., 0., 1., 0., 1.]),
 array([1., 0., 0., 0., 0., 1.]),
 array([1., 1., 0., 0., 1., 0.]),
 array([1., 1., 0., 1., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([1., 0., 1., 0., 0., 1.]),
 array([1., 0., 0., 1., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([1., 0., 1., 0., 0., 1.]),
 array([1., 0., 1., 0., 0., 0.]),
 array([1., 0., 1., 0., 0., 0.]),
 array([1., 0., 0., 0., 1., 1.]),
 array([0., 0., 0., 0., 0., 1.]),
 array([1., 0., 0., 1., 0., 1.]),
 array([0., 0., 0., 0., 0., 1.]),
 array([1., 1., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([1., 0., 0., 1., 0., 0.]),
 array([1., 0., 0., 0., 0., 1.]),
 array([1., 0., 1., 1., 0., 0.]),
 array([1., 0., 0., 0., 0., 1.]),
 array([1., 0., 0., 0., 0., 1.]),
 array([1., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1.]),
 array([1., 0.